In [2]:
import tensorflow as tf
devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

2023-09-01 10:41:21.388269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-01 10:41:21.430693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-01 10:41:21.431216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
import os
IMG_SHAPE = (256, 256)
N_CHANNELS = 1
INPUT_SHAPE = IMG_SHAPE + (N_CHANNELS,)
N_CLASSES = 1
OUTPUT_SHAPE = IMG_SHAPE + (N_CLASSES,)
BATCH_SIZE = 20
DATASET_PATH = f"{os.path.expanduser('~')}/Datasets/segthor_extracted"

In [11]:
import segmentation_models as sm
dice_score = sm.metrics.FScore(threshold=0.5)
precision_score = tf.keras.metrics.Precision(thresholds=0.5)
recall_score = tf.keras.metrics.Recall(thresholds=0.5)
iou_score = sm.metrics.IOUScore(threshold=0.5)

In [17]:
from model import EfficientUnet
model = EfficientUnet(input_shape = INPUT_SHAPE)
model.load_weights(f"records/{model.name}/checkpoint/{model.name}")

# Without Post Processing

In [18]:
import numpy as np
import preprocessing
from glob import glob
from generators import PerPatientSliceGenerator

preprocessing_pipeline = preprocessing.Pipeline([
    preprocessing.windowing(-500, 60),
    preprocessing.norm, 
    preprocessing.resize(IMG_SHAPE),
    preprocessing.expand_dims
])

metrics = {
    "dice":[],
    "precision":[],
    "recall":[],
    "iou_score":[],
}

patients_test = glob(DATASET_PATH + '/test/*')

for patient in patients_test:
    patient_dataset = tf.data.Dataset.from_generator(
        PerPatientSliceGenerator(patient, preprocessing_pipeline),
        output_signature=
        (
            tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
            tf.TensorSpec(shape=OUTPUT_SHAPE, dtype=tf.float32)
        )
    )

    volume_true = []
    for (x, y) in patient_dataset:
        volume_true.append(y)
    volume_true = np.squeeze(np.array(volume_true))
    volume_pred = model.predict(patient_dataset.batch(20))
    volume_pred = np.round(volume_pred)
    volume_pred = np.squeeze(volume_pred)

    metrics["dice"].append(dice_score(volume_true, volume_pred))
    metrics["precision"].append(precision_score(volume_true, volume_pred))
    metrics["recall"].append(recall_score(volume_true, volume_pred))
    metrics["iou_score"].append(iou_score(volume_true, volume_pred))
    
for k in metrics.keys():
    print(f"{k} : {np.mean(metrics[k])}")

['/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_07', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_06', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_10', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_08', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_11', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_05', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_09', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_03', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_01', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_04', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_02', '/home/arthur_guilherme/Datasets/segthor_extracted/test/Patient_12']
9/9 [==============================] - 1s 80ms/step
dice : 0.7283074259757996
precision : 0.6025879979133606
recall : 0.9349696040153503
iou_score : 0.5783645510673523


# With Post Processing

In [19]:
import numpy as np
import preprocessing
import postprocessing
from glob import glob
from generators import PerPatientSliceGenerator

preprocessing_pipeline = preprocessing.Pipeline([
    preprocessing.windowing(-500, 60),
    preprocessing.norm, 
    preprocessing.resize(IMG_SHAPE),
    preprocessing.expand_dims
])

metrics = {
    "dice":[],
    "precision":[],
    "recall":[],
    "iou_score":[],
}

patients_test = glob(DATASET_PATH + '/test/*')

for patient in patients_test:
    patient_dataset = tf.data.Dataset.from_generator(
        PerPatientSliceGenerator(patient, preprocessing_pipeline),
        output_signature=
        (
            tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
            tf.TensorSpec(shape=OUTPUT_SHAPE, dtype=tf.float32)
        )
    )

    volume_true = []
    for (x, y) in patient_dataset:
        volume_true.append(y)
    volume_true = np.squeeze(np.array(volume_true))
    volume_pred = model.predict(patient_dataset.batch(20))
    volume_pred = np.round(volume_pred)
    volume_pred = np.squeeze(volume_pred)
    volume_pred = postprocessing.biggest_3D_object(volume_pred)

    metrics["dice"].append(dice_score(volume_true, volume_pred))
    metrics["precision"].append(precision_score(volume_true, volume_pred))
    metrics["recall"].append(recall_score(volume_true, volume_pred))
    metrics["iou_score"].append(iou_score(volume_true, volume_pred))

for k in metrics.keys():
    print(f"{k} : {np.mean(metrics[k])}")

9/9 [==============================] - 1s 78ms/step
dice : 0.8052718043327332
precision : 0.6248044967651367
recall : 0.9329025745391846
iou_score : 0.6774531006813049
